In [1]:
# import time to count running time
import time
startTime = time.time()
# import pm2.5 value
weather_data = sc.textFile("hdfs://master:9000/user/datasets/pm25/pm25Taiwan.csv")

In [2]:
# split by comma
weather_data_rdd = weather_data.map(lambda line : line.split(","))
##############################print###############################
lis = weather_data_rdd.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

日期 測站 測項 00 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23
2015/01/01 龍潭 AMB_TEMP 14 14 14 13 13 13 12 12 13 14 14 14 14 14 13 13 12 11 11 11 11 11 11 11
2015/01/01 龍潭 CO 0.69 0.72 0.69 0.64 0.54 0.47 0.45 0.48 0.51 0.54 0.54 0.5 0.47 0.38 0.36 0.35 0.34 0.37 0.34 0.29 0.26 0.22 0.19 0.18


In [3]:
# remove not pm2.5, the value isn't decimal
import math
def remove_row_with_noise(x):
    for i in range(3, len(x)):
        if not x[i].isdecimal():
            return False
    return True

pm25_data = weather_data_rdd.filter(lambda x: x[2] == u"PM2.5")\
                     .filter(remove_row_with_noise)
    
# create every station name list (it will waste a lot of time, because of collect)
stationNames = pm25_data.map(lambda x:x[1]).distinct().collect()
##############################print###############################
lis = pm25_data.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print

for x in stationNames:
    print x,
print
##################################################################

2015/01/01 龍潭 PM2.5 46 71 76 74 65 62 56 50 52 56 54 47 40 36 37 27 30 25 26 24 18 16 11 14
2015/01/02 龍潭 PM2.5 15 12 9 14 17 20 18 22 21 23 18 25 24 27 18 23 18 19 18 21 23 18 19 19
2015/01/03 龍潭 PM2.5 14 14 9 13 12 15 9 11 14 27 27 24 16 27 38 39 35 32 38 36 34 36 36 39
龍潭 橋頭 汐止 竹山 士林 潮州 新莊 嘉義 永和 忠明 埔里 中山 斗六 頭份 前金 小港 彰化 萬里 苗栗 朴子 馬祖 南投 安南 竹東 善化 屏東 西屯 林口 基隆 馬公 三義 板橋 沙鹿 菜寮 三重 鳳山 古亭 新店 松山 前鎮 陽明 臺南 臺西 林園 美濃 桃園 臺東 大園 中壢 楠梓 關山 復興 平鎮 大同 觀音 新港 花蓮 麥寮 崙背 新營 土城 湖口 金門 淡水 大寮 宜蘭 仁武 萬華 新竹 豐原 左營 線西 恆春 冬山 大里 二林


In [4]:
# create the data style that we want
def Generated_Measurement(x):
    date = x[0]
    location = x[1]
    measure = x[2]
    measurements_of_a_day = []
    for i in range(3, len(x)):
        measurements_of_a_day.append((date, location, measure, "hr"+str(i-3), x[i]))
    return measurements_of_a_day

pm25_Measurement = pm25_data.flatMap(Generated_Measurement)
##############################print###############################
lis = pm25_Measurement.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

2015/01/01 龍潭 PM2.5 hr0 46
2015/01/01 龍潭 PM2.5 hr1 71
2015/01/01 龍潭 PM2.5 hr2 76


In [5]:
# arrange data, put date and hour together
pm25_KeyValue = pm25_Measurement.map(lambda x: ( (x[0], x[3]), x[1], x[4] ) )
##############################print###############################
lis = pm25_KeyValue.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
        print
##################################################################

(u'2015/01/01', 'hr0')
龍潭
46
(u'2015/01/01', 'hr1')
龍潭
71
(u'2015/01/01', 'hr2')
龍潭
76


In [6]:
# the same date and hour will put together
pm25_KeyValue_Group = pm25_KeyValue.groupBy(lambda x: x[0])
##############################print###############################
lis = pm25_KeyValue_Group.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

(u'2015/04/03', 'hr4') <pyspark.resultiterable.ResultIterable object at 0x7f4849b41ad0>
(u'2015/09/15', 'hr0') <pyspark.resultiterable.ResultIterable object at 0x7f4849b41710>
(u'2015/05/30', 'hr7') <pyspark.resultiterable.ResultIterable object at 0x7f4849b41b90>


In [7]:
# when we use the code "groupby", we should turn the group type to list
pm25_KeyValue_List = pm25_KeyValue_Group.mapValues(lambda x: list(x))
##############################print###############################
lis = pm25_KeyValue_List.first()
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

2015/04/03 hr4
((u'2015/04/03', 'hr4'), u'\u9f8d\u6f6d', u'17') ((u'2015/04/03', 'hr4'), u'\u9ea5\u5bee', u'19') ((u'2015/04/03', 'hr4'), u'\u9cf3\u5c71', u'1') ((u'2015/04/03', 'hr4'), u'\u99ac\u7956', u'18') ((u'2015/04/03', 'hr4'), u'\u982d\u4efd', u'19') ((u'2015/04/03', 'hr4'), u'\u95dc\u5c71', u'3') ((u'2015/04/03', 'hr4'), u'\u91d1\u9580', u'13') ((u'2015/04/03', 'hr4'), u'\u8c50\u539f', u'15') ((u'2015/04/03', 'hr4'), u'\u89c0\u97f3', u'15') ((u'2015/04/03', 'hr4'), u'\u897f\u5c6f', u'4') ((u'2015/04/03', 'hr4'), u'\u842c\u91cc', u'20') ((u'2015/04/03', 'hr4'), u'\u842c\u83ef', u'8') ((u'2015/04/03', 'hr4'), u'\u83dc\u5bee', u'25') ((u'2015/04/03', 'hr4'), u'\u82d7\u6817', u'21') ((u'2015/04/03', 'hr4'), u'\u82b1\u84ee', u'0') ((u'2015/04/03', 'hr4'), u'\u81fa\u897f', u'15') ((u'2015/04/03', 'hr4'), u'\u81fa\u6771', u'4') ((u'2015/04/03', 'hr4'), u'\u81fa\u5357', u'12') ((u'2015/04/03', 'hr4'), u'\u7f8e\u6fc3', u'2') ((u'2015/04/03', 'hr4'), u'\u7dda\u897f', u'11') ((u'2015/04/

In [8]:
# only left the date and hour contain Dali
def leftDali(x):
    for i in range(len(x[1])):
        if x[1][i][1] == u'大里':
            return True
    return False

pm25_KeyValue_List_Left_Dali = pm25_KeyValue_List.filter(leftDali)
##############################print###############################
lis = pm25_KeyValue_List_Left_Dali.first()
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

2015/04/03 hr4
((u'2015/04/03', 'hr4'), u'\u9f8d\u6f6d', u'17') ((u'2015/04/03', 'hr4'), u'\u9ea5\u5bee', u'19') ((u'2015/04/03', 'hr4'), u'\u9cf3\u5c71', u'1') ((u'2015/04/03', 'hr4'), u'\u99ac\u7956', u'18') ((u'2015/04/03', 'hr4'), u'\u982d\u4efd', u'19') ((u'2015/04/03', 'hr4'), u'\u95dc\u5c71', u'3') ((u'2015/04/03', 'hr4'), u'\u91d1\u9580', u'13') ((u'2015/04/03', 'hr4'), u'\u8c50\u539f', u'15') ((u'2015/04/03', 'hr4'), u'\u89c0\u97f3', u'15') ((u'2015/04/03', 'hr4'), u'\u897f\u5c6f', u'4') ((u'2015/04/03', 'hr4'), u'\u842c\u91cc', u'20') ((u'2015/04/03', 'hr4'), u'\u842c\u83ef', u'8') ((u'2015/04/03', 'hr4'), u'\u83dc\u5bee', u'25') ((u'2015/04/03', 'hr4'), u'\u82d7\u6817', u'21') ((u'2015/04/03', 'hr4'), u'\u82b1\u84ee', u'0') ((u'2015/04/03', 'hr4'), u'\u81fa\u897f', u'15') ((u'2015/04/03', 'hr4'), u'\u81fa\u6771', u'4') ((u'2015/04/03', 'hr4'), u'\u81fa\u5357', u'12') ((u'2015/04/03', 'hr4'), u'\u7f8e\u6fc3', u'2') ((u'2015/04/03', 'hr4'), u'\u7dda\u897f', u'11') ((u'2015/04/

In [9]:
# let the station tuple list become a big list
def Arrangement(x):
    arrangement_of_an_hour = []
    for i in range(0, len(x)):
        arrangement_of_an_hour.append(x[i][1])
        arrangement_of_an_hour.append(x[i][2])  
    return arrangement_of_an_hour

pm25_Arrangement = pm25_KeyValue_List_Left_Dali.mapValues(Arrangement)
##############################print###############################
lis = pm25_Arrangement.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

(u'2015/04/03', 'hr4') [u'\u9f8d\u6f6d', u'17', u'\u9ea5\u5bee', u'19', u'\u9cf3\u5c71', u'1', u'\u99ac\u7956', u'18', u'\u982d\u4efd', u'19', u'\u95dc\u5c71', u'3', u'\u91d1\u9580', u'13', u'\u8c50\u539f', u'15', u'\u89c0\u97f3', u'15', u'\u897f\u5c6f', u'4', u'\u842c\u91cc', u'20', u'\u842c\u83ef', u'8', u'\u83dc\u5bee', u'25', u'\u82d7\u6817', u'21', u'\u82b1\u84ee', u'0', u'\u81fa\u897f', u'15', u'\u81fa\u6771', u'4', u'\u81fa\u5357', u'12', u'\u7f8e\u6fc3', u'2', u'\u7dda\u897f', u'11', u'\u7af9\u6771', u'20', u'\u7af9\u5c71', u'13', u'\u6f6e\u5dde', u'4', u'\u6e56\u53e3', u'2', u'\u6de1\u6c34', u'21', u'\u6c99\u9e7f', u'5', u'\u6c50\u6b62', u'21', u'\u6a4b\u982d', u'4', u'\u6960\u6893', u'6', u'\u6843\u5712', u'26', u'\u6797\u5712', u'3', u'\u6797\u53e3', u'29', u'\u677f\u6a4b', u'24', u'\u677e\u5c71', u'14', u'\u6734\u5b50', u'7', u'\u65b0\u7af9', u'18', u'\u65b0\u71df', u'6', u'\u65b0\u6e2f', u'9', u'\u65b0\u5e97', u'16', u'\u6597\u516d', u'14', u'\u6046\u6625', u'5', u'\u5fe0\

In [10]:
# the Dali station and one of other stations put together
def putDaliWithOther(x):
    location_Dali = x.index(u"大里")
    compare_list = []
    for i in range(0,len(x),2):
        if x[i] != u'大里':
            compare_list.append((x[location_Dali], x[location_Dali+1], x[i], x[i+1]))
    return compare_list
pm25_Compare = pm25_Arrangement.mapValues(putDaliWithOther)
##############################print###############################
lis = pm25_Compare.first()
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

2015/04/03 hr4
(u'\u5927\u91cc', u'13', u'\u9f8d\u6f6d', u'17') (u'\u5927\u91cc', u'13', u'\u9ea5\u5bee', u'19') (u'\u5927\u91cc', u'13', u'\u9cf3\u5c71', u'1') (u'\u5927\u91cc', u'13', u'\u99ac\u7956', u'18') (u'\u5927\u91cc', u'13', u'\u982d\u4efd', u'19') (u'\u5927\u91cc', u'13', u'\u95dc\u5c71', u'3') (u'\u5927\u91cc', u'13', u'\u91d1\u9580', u'13') (u'\u5927\u91cc', u'13', u'\u8c50\u539f', u'15') (u'\u5927\u91cc', u'13', u'\u89c0\u97f3', u'15') (u'\u5927\u91cc', u'13', u'\u897f\u5c6f', u'4') (u'\u5927\u91cc', u'13', u'\u842c\u91cc', u'20') (u'\u5927\u91cc', u'13', u'\u842c\u83ef', u'8') (u'\u5927\u91cc', u'13', u'\u83dc\u5bee', u'25') (u'\u5927\u91cc', u'13', u'\u82d7\u6817', u'21') (u'\u5927\u91cc', u'13', u'\u82b1\u84ee', u'0') (u'\u5927\u91cc', u'13', u'\u81fa\u897f', u'15') (u'\u5927\u91cc', u'13', u'\u81fa\u6771', u'4') (u'\u5927\u91cc', u'13', u'\u81fa\u5357', u'12') (u'\u5927\u91cc', u'13', u'\u7f8e\u6fc3', u'2') (u'\u5927\u91cc', u'13', u'\u7dda\u897f', u'11') (u'\u5927\u9

In [11]:
# left couple of station
def getEveryValue(x):
    hour_list = []
    for i in range(len(x[1])):
        hour_list.append(x[1][i])
    return hour_list
pm25_AllValue = pm25_Compare.flatMap(getEveryValue)
##############################print###############################
lis = pm25_AllValue.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

大里 13 龍潭 17
大里 13 麥寮 19
大里 13 鳳山 1


In [12]:
# put the same couple of station together
def putToArray(x):
    daliArray = []
    otherArray = []
    for i in range(len(x)):
        daliArray.append(float(x[i][1]))
        otherArray.append(float(x[i][2]))
    return [daliArray,otherArray]
Complete_Compare_pm25 = pm25_AllValue.map(lambda x: ( (x[0], x[2]), x[1], x[3] ) )\
                        .groupBy(lambda x: x[0])\
                        .mapValues(lambda x: list(x))\
                        .mapValues(putToArray)
##############################print###############################
lis = Complete_Compare_pm25.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

(u'\u5927\u91cc', u'\u51ac\u5c71') [[58.0, 31.0, 38.0, 64.0, 12.0, 8.0, 53.0, 0.0, 33.0, 32.0, 18.0, 32.0, 11.0, 15.0, 37.0, 7.0, 43.0, 21.0, 32.0, 76.0, 56.0, 4.0, 26.0, 9.0, 39.0, 52.0, 43.0, 13.0, 45.0, 29.0, 27.0, 67.0, 57.0, 14.0, 2.0, 23.0, 42.0, 46.0, 49.0, 36.0, 30.0, 58.0, 44.0, 5.0, 27.0, 14.0, 37.0, 27.0, 28.0, 33.0, 15.0, 42.0, 32.0, 32.0, 12.0, 35.0, 32.0, 20.0, 35.0, 55.0, 25.0, 111.0, 90.0, 36.0, 20.0, 25.0, 21.0, 48.0, 31.0, 7.0, 31.0, 30.0, 19.0, 19.0, 1.0, 53.0, 43.0, 49.0, 1.0, 10.0, 33.0, 31.0, 22.0, 22.0, 33.0, 20.0, 12.0, 60.0, 0.0, 31.0, 22.0, 8.0, 23.0, 49.0, 5.0, 29.0, 61.0, 4.0, 7.0, 28.0, 12.0, 21.0, 83.0, 23.0, 43.0, 26.0, 70.0, 28.0, 17.0, 23.0, 34.0, 49.0, 79.0, 43.0, 23.0, 51.0, 6.0, 43.0, 62.0, 53.0, 51.0, 56.0, 38.0, 30.0, 23.0, 12.0, 9.0, 0.0, 12.0, 71.0, 53.0, 72.0, 75.0, 32.0, 0.0, 38.0, 87.0, 5.0, 16.0, 99.0, 31.0, 27.0, 64.0, 20.0, 36.0, 17.0, 25.0, 54.0, 86.0, 33.0, 17.0, 33.0, 51.0, 1.0, 59.0, 9.0, 14.0, 45.0, 28.0, 70.0, 7.0, 50.0, 43.0, 36.0, 4

In [13]:
# Calculate Correlation
import numpy as np
def CalculateCorrelation(x):
    corr = np.corrcoef(x[0], x[1])
    return corr[0][1]
pm25_Corr = Complete_Compare_pm25.mapValues(CalculateCorrelation).map(lambda x:(x[1],x[0][1]))
##############################print###############################
lis = pm25_Corr.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

0.327624899393 冬山
0.477576173148 平鎮
0.464566398445 桃園


In [14]:
# sort the correlation and print the highest 3 station
result = pm25_Corr.sortByKey(ascending=False)
##############################print###############################
lis = result.take(3)
for x in lis:
    for i in range(len(x)):
        print x[i],
    print
##################################################################

0.880213058015 忠明
0.858951681025 西屯
0.830540673781 南投


In [15]:
# print the total running time
endTime = time.time()
print (endTime-startTime)

18.6231188774
